<a href="https://colab.research.google.com/github/lewiczmichal/car-price-prediction/blob/main/Car_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 -m pip install --upgrade tables
!python3 -m pip install eli5
!python3 -m pip install xgboost
!python3 -m pip install hyperopt
!pip install scikit-plot
!pip install catboost

In [2]:
import pandas as pd
import numpy as np
np.random.seed(0)

from collections import defaultdict
import gc
import re

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error
from scikitplot.estimators import plot_learning_curve

from functools import partial
from hyperopt import hp
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

from tqdm import tqdm

import xgboost as xgb
import catboost as ctb

pd.set_option('display.float_format', '{:.5f}'.format) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
%cd '/content/drive/My Drive/Colab Notebooks/Konkurs'

/content/drive/My Drive/Colab Notebooks/Konkurs


In [5]:
train = pd.read_hdf('train.car_price.h5')
test = pd.read_hdf('test.car_price.h5') 

In [6]:
train.drop('price_details', axis=1, inplace=True)
train.loc[52957, 'param_liczba-pozostałych-rat'] = -1
test.loc[50910, 'param_liczba-pozostałych-rat'] = -1
train.loc[57222, 'param_przebieg'] = 1

#Przemnożenie cen w Euro
train['price_value'] = np.where(train['price_currency']=='EUR', train['price_value'].map(lambda x: x*4), train['price_value'].map(lambda x: x))

# Preprocessing danych

In [7]:
def clean_data(df):
    
    #Uzupełnienie brakujących wartości z cech z angielską nazwą
    df['param_kolor'].fillna(df['param_color'], inplace=True)
    df['param_kraj-pochodzenia'].fillna(df['param_country-of-origin'], inplace=True)
    df['param_rok-produkcji'].fillna(df['param_year'], inplace=True)
    df['param_liczba-miejsc'].fillna(df['param_nr-of-seats'], inplace=True)
    df['param_uszkodzony'].fillna(df['param_damaged'], inplace=True)
    df['param_przebieg'].fillna(df['param_mileage'], inplace=True)
    df['param_liczba-drzwi'].fillna(df['param_door-count'], inplace=True)
    df['param_skrzynia-biegów'].fillna(df['param_gearbox'], inplace=True)
    df['param_bezwypadkowy'].fillna(df['param_no-accident'], inplace=True)
    df['param_wersja'].fillna(df['param_version'], inplace=True)
    df['param_metalik'].fillna(df['param_metallic'], inplace=True)
    df['param_moc'].fillna(df['param_engine-power'], inplace=True) 
    df['param_rodzaj-paliwa'].fillna(df['param_fuel-type'], inplace=True)
    df['param_perłowy'].fillna(df['param_pearl'], inplace=True)
    df['param_filtr-cząstek-stałych'].fillna(df['param_particle-filter'], inplace=True)
    df['param_pierwszy-właściciel'].fillna(df['param_original-owner'], inplace=True)
    df['param_pojemność-skokowa'].fillna(df['param_engine-capacity'], inplace=True)
    df['param_akryl-(niemetalizowany)'].fillna(df['param_acrylic'], inplace=True)
    df['param_miesięczna-rata'].fillna(df['param_monthly-payment-value'], inplace=True)
    df['param_model-pojazdu'].fillna(df['param_model'], inplace=True)
    df['param_leasing'].fillna(df['param_leasing-concession'], inplace=True)
    df['param_typ'].fillna(df['param_body-type'], inplace=True)
    df['param_marka-pojazdu'].fillna(df['param_make'], inplace=True)
    df['param_napęd'].fillna(df['param_transmission'], inplace=True)
    df['param_zarejestrowany-w-polsce'].fillna(df['param_registered-in-poland'], inplace=True)
    df['param_możliwość-finansowania'].fillna(df['param_financing-option'], inplace=True)
    df['param_serwisowany-w-aso'].fillna(df['param_service-record'], inplace=True)
    df['param_kierownica-po-prawej-(anglik)'].fillna(df['param_tuning'], inplace=True)
    df['seller_type'] = df['seller_type'].map(lambda x: 'Firmy' if x=='Dealer' else ('Osoby prywatnej' if x=='Osoba prywatna' else x))
    df['param_oferta-od'].fillna(df['seller_type'], inplace=True)
    
    #Uzupełnienie brakujących wartości z kolumny breadcrumb
    for i in enumerate(df['param_marka-pojazdu']):
        if i[1] == None:
            df['param_marka-pojazdu'].iloc[i[0]] = df['breadcrumb'].iloc[i[0]][1]
    
    for i in enumerate(df['param_model-pojazdu']):
        if i[1] == None:
            df['param_model-pojazdu'].iloc[i[0]] = df['breadcrumb'].iloc[i[0]][2]
    
    #Usuwanie niepotrzebnych już cech
    cols_to_delete = ['param_color', 'param_country-of-origin', 'param_year', 'param_nr-of-seats', 'param_damaged',
                  'param_co2-emissions', 'param_mileage', 'param_door-count', 'param_gearbox', 'param_no-accident',
                  'param_version', 'param_metallic', 'param_engine-power', 'param_fuel-type', 'param_pearl', 
                  'param_particle-filter', 'param_original-owner', 'param_engine-code', 'param_engine-capacity',
                  'param_acrylic', 'param_monthly-payment-value', 'param_model', 'param_leasing-concession', 
                  'param_body-type', 'param_make', 'param_transmission', 'param_registered-in-poland',
                  'param_financing-option', 'param_service-record', 'param_tuning', 'created_at',
                  'param_pierwsza-rejestracja', 'param_first-registration', 'param_vat-free', 
                  'param_vat-discount', 'breadcrumb', 'param_emisja-co2', 'seller_type', 
                  'param_kategoria']
    
    df.drop(cols_to_delete, axis=1, inplace=True)
    
    df.fillna(-1, inplace=True)
    
    
    #Konwertowanie wartości kategorialnych na int 
    df['param_liczba-miejsc'] = df['param_liczba-miejsc'].astype(int)
    df['param_rok-produkcji'] = df['param_rok-produkcji'].astype(int)
    df['param_liczba-drzwi'] = df['param_liczba-drzwi'].astype(int).map(lambda x: 5 if x > 5 else x)
    df['param_przebieg'] = df['param_przebieg'].astype(str).map(lambda x: x.replace(' km', '').replace(' ', '')).astype(int)
    df['param_wartość-wykupu'] = df['param_wartość-wykupu'].astype(str).map(lambda x: x.replace(' PLN', '')\
                                                                 .replace(' ', '').replace(',', '')).astype(int)\
                                                                 .map(lambda x: -1 if x>300000 else x)
    df['param_moc'] = df['param_moc'].astype(str).map(lambda x: x.replace(' KM', '').replace(' HP', '')\
                                                   .replace(' ', '')).astype(int).map(lambda x: -1 if x>1000 else x)
    df['param_miesięczna-rata'] = df['param_miesięczna-rata'].astype(str).map(lambda x: x.replace(' PLN', '').replace(' ', '')\
                                                   .replace(',', '')).astype(int).map(lambda x: -1 if x>50000 else x)
    df['param_opłata-początkowa'] = df['param_opłata-początkowa'].astype(str).map(lambda x: x.replace(' PLN', '').replace(' ', '')\
                                                   .replace(',', '')).astype(int).map(lambda x: -1 if x>500000 else x)
    df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].astype(str).map(lambda x: x.replace(' cm3', '').replace(' ', ''))\
                                                               .astype(int).map(lambda x: -1 if x>6000 else x)
    

    df.fillna(-1, inplace=True)
    
    df['param_uszkodzony'] = df['param_uszkodzony'].map(lambda x: 0 if x==-1 else 1)
    df['param_faktura-vat'] = df['param_faktura-vat'].map(lambda x: 0 if x==-1 else 1)
    df['param_perłowy'] = df['param_perłowy'].map(lambda x: 0 if x==-1 else 1)
    df['param_homologacja-ciężarowa'] = df['param_homologacja-ciężarowa'].map(lambda x: 0 if x==-1 else 1)
    df['param_vat-marża'] = df['param_vat-marża'].map(lambda x: 0 if x==-1 else 1)
    df['param_leasing'] = df['param_leasing'].map(lambda x: 0 if x==-1 else 1)
    df['param_zarejestrowany-jako-zabytek'] = df['param_zarejestrowany-jako-zabytek'].map(lambda x: 0 if x==-1 else 1)
    df['param_metalik'] = df['param_metalik'].map(lambda x: 0 if x==-1 else 1)
    df['param_pierwszy-właściciel'] = df['param_pierwszy-właściciel'].map(lambda x: 0 if x==-1 else 1)
    df['param_zarejestrowany-w-polsce'] = df['param_zarejestrowany-w-polsce'].map(lambda x: 0 if x==-1 else 1)
    df['param_serwisowany-w-aso'] = df['param_serwisowany-w-aso'].map(lambda x: 0 if x==-1 else 1)
    df['param_możliwość-finansowania'] = df['param_możliwość-finansowania'].map(lambda x: 0 if x==-1 else 1)
    df['param_kierownica-po-prawej-(anglik)'] = df['param_kierownica-po-prawej-(anglik)'].map(lambda x: 0 if x==-1 else 1)
    df['param_filtr-cząstek-stałych'] = df['param_filtr-cząstek-stałych'].map(lambda x: 0 if x==-1 else 1)
    df['param_stan'] = df['param_stan'].map(lambda x: 0 if x=='Używane' else (1 if x=='Nowe' else -1))
    df['param_oferta-od'] = df['param_oferta-od'].map(lambda x: 0 if x=='Osoby prywatnej' else (1 if x=='Firmy' else -1))
    df['param_matowy'] = df['param_matowy'].map(lambda x: 0 if x==-1 else 1)
    df['param_bezwypadkowy'] = df['param_bezwypadkowy'].map(lambda x: 0 if x==-1 else 1)
    df['param_akryl-(niemetalizowany)'] = df['param_akryl-(niemetalizowany)'].map(lambda x: 0 if x==-1 else 1)
    df['param_liczba-pozostałych-rat'] = df['param_liczba-pozostałych-rat'].map(lambda x: -1 if x==None else x).astype(int)

In [8]:
clean_data(train)
clean_data(test)

In [9]:
#Połączenie train i test w celu późniejszejszego zastosowania Label Encoding
dataset = pd.concat(objs=[train, test], axis=0)
del train, test
gc.collect()

0

# Feature Engineering

In [10]:
def num_filter(val):
    return re.sub('[^0-9]','', str(val))

In [11]:
#Analiza roku produkcji danej wersji samochodu
def find_first_date(row):
    if row != -1:
        first_year = re.findall("([0-9]{4})-", row)
        if len(first_year) > 0:
            return int(first_year[0])
    return -1

def find_last_date(row):
    if row != -1:
        first_year = re.findall("-([0-9]{4})", row)
        if len(first_year) > 0:
            return int(first_year[0])
        return 2018
    return -1

dataset["start_produce"] = dataset["param_wersja"].map(find_first_date)
dataset["finish_produce"] = dataset["param_wersja"].map(find_last_date)

def find_difference_first_year(row):
    if row["param_rok-produkcji"] == -1:
        return -10
    elif row["start_produce"] != -1:
        return row["param_rok-produkcji"] - row["start_produce"]

dataset["age_of_model"] = dataset.apply(find_difference_first_year, axis=1).fillna(-1).astype('int8')

In [12]:
def feature_engineering(df):
    df['is_EUR'] = df['price_currency'].map(lambda x: int(x == 'EUR'))
    df['is_vin'] = df['param_vin'].map(lambda x: int(x != -1))
    df.drop(['param_vin'], axis=1, inplace=True)
    df['is_white'] = df['param_kolor'].map(lambda x: int(x=='Biały'))
    df['is_black'] = df['param_kolor'].map(lambda x: int(x=='Czarny'))
    df['is_silver'] = df['param_kolor'].map(lambda x: int(x=='Srebrny'))
    
    df['is_Audi'] = df['param_marka-pojazdu'].map(lambda x: int(x=='Audi'))
    df['is_Volkswagen'] = df['param_marka-pojazdu'].map(lambda x: int(x=='Volkswagen'))
    df['is_BMW'] = df['param_marka-pojazdu'].map(lambda x: int(x=='BMW'))
    df['is_Mercedes-Benz'] = df['param_marka-pojazdu'].map(lambda x: int(x=='Mercedes-Benz'))
    df['is_Ferrari'] = df['param_marka-pojazdu'].map(lambda x: int(x=='Ferrari'))
    df['is_Rolls-Royce'] = df['param_marka-pojazdu'].map(lambda x: int(x=='Rolls-Royce'))
    df['is_Bentley'] = df['param_marka-pojazdu'].map(lambda x: int(x=='Bentley'))
    df['is_AMG_GT'] = df['param_model-pojazdu'].map(lambda x: int(x=='AMG GT'))
    df['is_Supra'] = df['param_model-pojazdu'].map(lambda x: int(x=='Supra'))
    df['is_R231 (2012-)'] = df['param_wersja'].map(lambda x: int(x=='R231 (2012-)'))
     
    
    bmw_or_audi_or_merc = ['BMW', 'Audi', 'Mercedes-Benz']
    df['is_BMW_or_Audi_or_Merc'] = df['param_marka-pojazdu'].map(lambda x: 1 if x in bmw_or_audi_or_merc else 0)
    
    df['is_Ferrari_Warszawa'] = df['seller_name'].map(lambda x: int(x=='Ferrari Warszawa'))
    df['is_Multi_Cars'] = df['seller_name'].map(lambda x: int(x=='Multi Cars - Samochody Luksusowe i Jachty'))
    df['is_Luxury-Select-Car'] = df['seller_name'].map(lambda x: int(x=='Luxury-Select Car'))
    df['is_Pn_Wlkp_Nowe_Miasto'] = df['seller_address'].map(lambda x: int(x=='Poznań, Wielkopolskie, Nowe Miasto'))
    
    klasa = ['Klasa S', 'Klasa G', 'GLS', 'GLE', 'Klasa E']
    df['is_Klasa_S_G_GLS_GLE_E'] = df['param_model-pojazdu'].map(lambda x: 1 if x in klasa else 0)
    
    bmw_or_audi_or_merc = ['BMW', 'Audi', 'Mercedes-Benz']
    df['is_BMW_or_Audi_or_Merc'] = df['param_marka-pojazdu'].map(lambda x: 1 if x in bmw_or_audi_or_merc else 0)
    
    cheap_cars = ['Daewoo', 'Daihatsu', 'Polonez', 'Rover', 'Saab', 'Smart', 'Wartburg']
    mid_cars = ['Citroën', 'Fiat', 'Honda', 'Mazda', 'Mitsubishi', 'Renault', 'Suzuki', 'Seat']
    luxury_cars = ['Ferrari', 'Bentley', 'Porsche', 'Aston Martin', 'Maserati', 'Rolls-Royce', 'McLaren', 'Lincoln',
               'Lamborghini', 'Tesla']

    df['is_luxury_car'] = df['param_marka-pojazdu'].map(lambda x: 1 if x in luxury_cars else 0)
    df['is_cheap_car'] = df['param_marka-pojazdu'].map(lambda x: 1 if x in cheap_cars else 0)
    df['is_mid_car'] = df['param_marka-pojazdu'].map(lambda x: 1 if x in mid_cars else 0)
    
    df['is_faktura_and_leasing'] = ((df['param_faktura-vat'] == 1) & (df['param_leasing'] == 1)).astype(int)
    
    df['is_older_than_1960'] = (df['param_rok-produkcji'] < 1960).astype(int)
    df['is_1960-1970'] = ((df['param_rok-produkcji'] > 1959) & (df['param_rok-produkcji'] < 1971)).astype(int)
    df['is_2006-2010'] = ((df['param_rok-produkcji'] > 2005) & (df['param_rok-produkcji'] < 2011)).astype(int)
    df['is_2011-2015'] = ((df['param_rok-produkcji'] > 2010) & (df['param_rok-produkcji'] < 2016)).astype(int)
    df['is_over_2015'] = (df['param_rok-produkcji'] > 2015).astype(int)
    
    df['is_100-300KM'] = ((df['param_moc'] > 99) & (df['param_moc'] < 301)).astype(int)
    df['is_300-500KM'] = ((df['param_moc'] > 299) & (df['param_moc'] < 501)).astype(int)
    df['is_over_500KM'] = (df['param_moc'] > 499).astype(int)
    
    df['is_2000-3000cm2'] = ((df['param_pojemność-skokowa'] > 1999) & (df['param_pojemność-skokowa'] < 3001)).astype(int)
    df['is_3000-4000cm2'] = ((df['param_pojemność-skokowa'] > 2999) & (df['param_pojemność-skokowa'] < 4001)).astype(int)
    df['is_over_4000cm2'] = (df['param_pojemność-skokowa'] > 4000).astype(int)

    df['is_over_500KM_and_2015'] = ((df['param_moc'] > 499) & (df['param_rok-produkcji'] > 2014)).astype(int)
    df['is_over_500KM_and_<50000km'] = ((df['param_moc'] > 499) & (df['param_przebieg'] < 50000)).astype(int)
    df['is_less_30000km_and_2017'] = ((df['param_przebieg'] < 30000) & (df['param_rok-produkcji'] > 2016)).astype(int)
    
    df['przebieg_less_than_100km'] = (df['param_przebieg'] < 101).astype(int)
    df['is_100-20000km'] = ((df['param_przebieg'] > 99) & (df['param_przebieg'] < 20001)).astype(int)
    df['is_20000-100000km'] = ((df['param_przebieg'] > 19999) & (df['param_przebieg'] < 100001)).astype(int)
    df['is_100000-200000km'] = ((df['param_przebieg'] > 99999) & (df['param_przebieg'] < 200001)).astype(int)
    df['is_over_200000km'] = (df['param_przebieg'] > 200000).astype(int)
    df['is_5000000km'] = (df['param_przebieg'] == 5000000).astype(int)
    
    #Dodanie mniej licznych wartości w jedną wartość other
    top_countries = list(df['param_kraj-pochodzenia'].value_counts().head(17).keys())
    country_norm = defaultdict(lambda: 'other_country', dict(zip(top_countries, top_countries)))
    df['param_kraj'] = df['param_kraj-pochodzenia'].map(country_norm)
    
    top_make = list(df['param_marka-pojazdu'].value_counts().head(55).keys())
    make_norm = defaultdict(lambda: 'other_make', dict(zip(top_make, top_make)))
    df['param_marka'] = df['param_marka-pojazdu'].map(make_norm)
    
    top_type = list(df['param_typ'].value_counts().head(9).keys())
    type_norm = defaultdict(lambda: 'other_type', dict(zip(top_type, top_type)))
    df['param_typ'] = df['param_typ'].map(type_norm)
    
    top_model = list(df['param_model-pojazdu'].value_counts().head(300).keys())
    model_norm = defaultdict(lambda: 'other_model', dict(zip(top_model, top_model)))
    df['param_model'] = df['param_model-pojazdu'].map(model_norm)
    
    top_version = list(df['param_wersja'].value_counts().head(250).keys())
    version_norm = defaultdict(lambda: 'other_version', dict(zip(top_version, top_version)))
    df['param_wersja'] = df['param_wersja'].map(version_norm)
    
    top_seller = list(df['seller_name'].value_counts().head(500).keys())
    seller_norm = defaultdict(lambda: 'other_seller', dict(zip(top_seller, top_seller)))
    df['seller_name'] = df['seller_name'].map(seller_norm)
    
    top_address = list(df['seller_address'].value_counts().head(200).keys())
    address_norm = defaultdict(lambda: 'other_address', dict(zip(top_address, top_address)))
    df['seller_address'] = df['seller_address'].map(address_norm)
    
    top_code = list(df['param_kod-silnika'].value_counts().head(115).keys())
    code_norm = defaultdict(lambda: 'other_code', dict(zip(top_code, top_code)))
    df['param_kod-silnika'] = df['param_kod-silnika'].map(code_norm)
    
    cols_to_delete = ['param_marka-pojazdu', 'param_kraj-pochodzenia', 'param_model-pojazdu']
    df.drop(cols_to_delete, axis=1, inplace=True)
    
    cat_feats = ['param_napęd', 'param_skrzynia-biegów', 'param_rodzaj-paliwa', 'param_kolor', 'param_kraj', 
                 'param_marka', 'param_typ', 'param_model', 'param_wersja', 'seller_name', 'seller_address', 'param_kod-silnika']
    for cat_feat in cat_feats:
        df['{0}_cat'.format(cat_feat)] = pd.factorize(df[cat_feat])[0]

    df.drop(cat_feats, axis=1, inplace=True)
    
    df['param_moc_norm'] = np.log(df['param_moc'].map(num_filter).astype(np.int))
    df['param_pojemność-skokowa_norm'] = np.log(df['param_pojemność-skokowa'].map(num_filter).astype(np.int))
    df['param_przebieg_norm'] = np.log(df['param_przebieg'] + 1000)
    
    #Liczba wartości null dla danego wiersza
    df['null_count'] = df.eq(-1).sum(axis=1)
    
    #Liczba wszystkich cech feature dla danego wiersza
    bool_feats = df.select_dtypes(include=[np.bool]).columns
    df['bool_feats_count'] = df[bool_feats].eq(True).sum(axis=1)
    
    #Liczba znaczących cech feature dla danego wiersza
    key_feats = ['feature_gniazdo-usb', 'feature_kamera-cofania', 'feature_czujniki-parkowania-tylne', 
                  'feature_nawigacja-gps', 'feature_bluetooth', 'feature_światła-do-jazdy-dziennej', 'feature_światła-led', 
                  'feature_czujniki-parkowania-przednie']
    df['key_feats_count'] = df[key_feats].eq(True).sum(axis=1)
    
    #Średnie wartości dla wierszy o dużym błędzie
    error_features_true = ['feature_elektrycznie-ustawiane-lusterka', 'feature_elektryczne-szyby-przednie',
                           'feature_asr-(kontrola-trakcji)', 'feature_poduszka-powietrzna-pasażera', 'feature_komputer-pokładowy', 
                           'feature_klimatyzacja-manualna', 'feature_alufelgi', 'feature_wspomaganie-kierownicy', 
                           'feature_radio-fabryczne', 'feature_centralny-zamek', 'feature_immobilizer', 'feature_abs',
                           'feature_esp-(stabilizacja-toru-jazdy)', 'feature_czujnik-deszczu']
    error_features_false = ['feature_ogrzewanie-postojowe', 'feature_radio-niefabryczne', 'feature_tuner-tv']
    df['error_features_true'] = df[error_features_true].eq(True).sum(axis=1)
    df['error_features_false'] = df[error_features_false].eq(False).sum(axis=1)
    

In [13]:
feature_engineering(dataset)

In [14]:
#Optymalizacja pamięci
for feat in dataset.select_dtypes(include=['int']):
    if dataset[feat].max() < np.iinfo(np.int8).max: 
        dataset[feat] = dataset[feat].astype('int8')
    elif dataset[feat].max() < np.iinfo(np.int16).max:
        dataset[feat] = dataset[feat].astype('int16')
    elif dataset[feat].max() < np.iinfo(np.int32).max:
        dataset[feat] = dataset[feat].astype('int32')

for feat in dataset.select_dtypes(include=['float']):
    if dataset[feat].max() < np.finfo(np.float16).max: 
        dataset[feat] = dataset[feat].astype('float16')
    elif dataset[feat].max() < np.finfo(np.float16).max: 
        dataset[feat] = dataset[feat].astype('float32')

In [15]:
#Rozdzielenie zbiorów train i test
train = dataset[~dataset.price_value.isnull()].copy()
test = dataset[dataset.price_value.isnull()].copy()
del dataset
gc.collect()

0

# Analiza błędów predykcji

In [16]:
black_list = ['car_id', 'price_details', 'price_value', 'price_currency']
feats = train.select_dtypes(include=[np.number, np.bool]).columns
feats = [x for x in feats if x not in black_list]

In [17]:
train.reset_index(drop=True, inplace=True)
X = train[feats].values
y = train['price_value']
y_log = np.log(y)

In [20]:
ctb_params = {'border_count': 195,
              'depth': 6, 
              'iterations': 1000, 
              'l2_leaf_reg': 5.885578634980554, 
              'learning_rate': 0.06301319304159513, 
              'min_data_in_leaf': 37, 
              'random_strength': 1.984062456758668}

In [ ]:
price_value_pred = {}
cv = KFold(n_splits=5)

for train_idx, test_idx in tqdm(cv.split(X, y_log)):
    X_train, y_log_train = X[train_idx], y_log[train_idx]
    X_test, y_log_test   = X[test_idx], y_log[test_idx]
    
    model = ctb.CatBoostRegressor(**ctb_params)
    model.fit(X_train, y_log_train)
    y_log_pred = model.predict(X_test)
    
    
    y_pred = np.exp(y_log_pred)
    global_min = y.min()
    y_pred[ y_pred < global_min] = global_min

    price_value_pred.update( dict(zip(list(test_idx), list(y_pred))) )
    
train['price_value_pred'] = train.index.map(price_value_pred)

In [22]:
train['pred_diff'] = train['price_value'] - train['price_value_pred']
train['pred_diff_abs'] = train['pred_diff'].abs()

In [23]:
df = train[ ['price_value', 'price_value_pred', 'pred_diff_abs'] ].sort_values(by='pred_diff_abs', ascending=False).head(25000)

In [24]:
error_rows = []
for row in df.index: 
    error_rows.append(row)

In [25]:
err = train.iloc[error_rows]

In [26]:
#Sprawdzanie średnich wartości dla features w pierwszych 25000 wierszy o największym błędzie
err.mean(axis=0).to_frame().T

,price_value,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,feature_wspomaganie-kierownicy,feature_podgrzewana-przednia-szyba,feature_przyciemniane-szyby,feature_elektrycznie-ustawiane-fotele,feature_klimatyzacja-czterostrefowa,feature_tuner-tv,feature_poduszki-boczne-przednie,...,is_2011-2015,is_over_2015,is_100-300KM,is_300-500KM,is_over_500KM,is_2000-3000cm2,is_3000-4000cm2,is_over_4000cm2,is_over_500KM_and_2015,is_over_500KM_and_<50000km,is_less_30000km_and_2017,przebieg_less_than_100km,is_100-20000km,is_20000-100000km,is_100000-200000km,is_over_200000km,is_5000000km,param_napęd_cat,param_skrzynia-biegów_cat,param_rodzaj-paliwa_cat,param_kolor_cat,param_kraj_cat,param_marka_cat,param_typ_cat,param_model_cat,param_wersja_cat,seller_name_cat,seller_address_cat,param_kod-silnika_cat,param_moc_norm,param_pojemność-skokowa_norm,param_przebieg_norm,null_count,bool_feats_count,key_feats_count,error_features_true,error_features_false,price_value_pred,pred_diff,pred_diff_abs
0,108789.02387,0.48932,0.30236,0.62016,0.47988,0.45860,0.57800,0.83704,0.71184,0.82532,0.73020,0.85256,0.82292,0.41388,0.57692,0.75132,0.66120,0.15756,0.16532,0.60876,0.61804,0.05372,0.03800,0.10076,0.26956,0.11760,0.14552,0.62452,0.17864,0.75252,0.66224,0.52380,0.20840,0.83228,0.19440,0.45204,0.35020,0.08788,0.02976,0.62284,...,0.34636,0.42896,0.76136,0.09228,0.01492,0.32728,0.08844,0.04416,0.00768,0.00844,0.31892,0.23540,0.15308,0.25684,0.25152,0.11000,0.00040,1.94332,1.02936,1.45104,3.68848,1.63280,13.42388,2.23904,94.12188,63.76112,68.77988,21.88940,6.63468,inf,inf,inf,5.31980,31.96936,4.24644,10.40536,2.87852,103916.18681,4872.83706,21151.56340


# Feature Selection

In [27]:
#Wybranie 20% danych dla przyśpieszenia obliczeń
train_light = train.sample(frac=0.2)
X_light = train_light[feats].values
y_light_log = np.log1p(train_light['price_value'].values)

In [28]:
import eli5
from eli5.sklearn import PermutationImportance

In [31]:
def eli_feats(feats):
    
    ctb_params = {'border_count': 195,
                  'depth': 6, 
                  'iterations': 1000, 
                  'l2_leaf_reg': 5.885578634980554, 
                  'learning_rate': 0.06301319304159513, 
                  'min_data_in_leaf': 37, 
                  'random_strength': 1.984062456758668,
                  'verbose' : False}
    
    train_light = train.sample(frac=0.1)
    X_light = train_light[feats].values
    y_light_log = np.log1p(train_light['price_value'].values)
    
    X_sub_train, X_sub_test, y_sub_train, y_sub_test = train_test_split(X_light, y_light_log, shuffle=True)
    
    model = ctb.CatBoostRegressor(**ctb_params).fit(X_sub_train, y_sub_train)
    perm = PermutationImportance(model, random_state=1).fit(X_sub_test, y_sub_test)
    
    return eli5.explain_weights(perm, feature_names = feats, top=150)

In [32]:
eli_feats(feats)

Weight,Feature
0.5956 ± 0.0206,param_rok-produkcji
0.0253 ± 0.0006,param_moc_norm
0.0222 ± 0.0036,param_uszkodzony
0.0165 ± 0.0025,param_typ_cat
0.0120 ± 0.0025,param_marka_cat
0.0101 ± 0.0018,param_przebieg_norm
0.0100 ± 0.0024,param_moc
0.0077 ± 0.0023,param_napęd_cat
0.0068 ± 0.0016,is_BMW_or_Audi_or_Merc
0.0063 ± 0.0017,param_przebieg


In [33]:
feat_for_split = ['param_rok-produkcji', 'param_moc', 'param_przebieg', 'param_uszkodzony', 'param_typ_cat', 'bool_feats_count', 'param_marka_cat',
                  'param_pojemność-skokowa', 'is_BMW_or_Audi_or_Merc', 'param_napęd_cat', 'key_feats_count', 'param_skrzynia-biegów_cat', 'finish_produce', 
                  'start_produce', 'is_mid_car', 'age_of_model', 'param_model_cat', 'param_wersja_cat', 'param_liczba-miejsc', 'param_bezwypadkowy',
                  'is_Volkswagen', 'param_faktura-vat', 'param_kraj_cat', 'is_luxury_car', 'param_rodzaj-paliwa_cat', 'param_kierownica-po-prawej-(anglik)',
                  'is_2011-2015', 'param_pojemność-skokowa_norm', 'is_Audi', 'feature_tapicerka-welurowa', 'seller_name_cat', 'feature_alufelgi', 
                  'param_przebieg_norm', 'is_over_2015', 'feature_elektryczne-szyby-tylne', 'param_liczba-drzwi', 'feature_światła-led',
                  'param_możliwość-finansowania', 'is_cheap_car', 'param_kolor_cat', 'param_oferta-od', 'error_features_true', 'is_100-300KM', 'null_count',
                  'param_serwisowany-w-aso', 'is_2006-2010', 'is_Klasa_S_G_GLS_GLE_E', 'feature_klimatyzacja-manualna', 'feature_czujniki-parkowania-przednie',
                  'param_zarejestrowany-w-polsce', 'feature_kamera-cofania', 'feature_klimatyzacja-automatyczna', 'feature_esp-(stabilizacja-toru-jazdy)', 
                  'feature_tapicerka-skórzana', 'is_vin', 'feature_poduszka-powietrzna-kierowcy', 'feature_podgrzewane-przednie-siedzenia', 
                  'feature_elektrycznie-ustawiane-fotele', 'feature_regulowane-zawieszenie', 'feature_przyciemniane-szyby', 'param_moc_norm', 'is_silver', 
                  'feature_gniazdo-usb', 'feature_hud-(wyświetlacz-przezierny)', 'param_metalik', 'feature_wielofunkcyjna-kierownica', 'feature_czujnik-deszczu', 
                  'feature_światła-xenonowe', 'param_kod-silnika_cat', 'feature_dach-panoramiczny', 'feature_tempomat', 'is_faktura_and_leasing', 
                  'feature_klimatyzacja-czterostrefowa', 'feature_bluetooth', 'is_BMW', 'feature_kurtyny-powietrzne', 'param_akryl-(niemetalizowany)', 
                  'feature_komputer-pokładowy', 'feature_cd', 'param_miesięczna-rata', 'feature_światła-do-jazdy-dziennej', 'feature_klimatyzacja-dwustrefowa', 
                  'feature_elektrycznie-ustawiane-lusterka', 'param_vat-marża', 'feature_elektrochromatyczne-lusterko-wsteczne', 'param_homologacja-ciężarowa', 
                  'is_Mercedes-Benz', 'feature_poduszka-powietrzna-chroniąca-kolana', 'feature_system-start-stop', 'is_100-20000km', 'feature_asr-(kontrola-trakcji)',
                  'feature_elektrochromatyczne-lusterka-boczne', 'feature_gniazdo-aux', 'feature_czujniki-parkowania-tylne', 'is_100000-200000km',
                  'feature_centralny-zamek', 'feature_elektryczne-szyby-przednie', 'feature_ogrzewanie-postojowe', 'feature_immobilizer',
                  'param_filtr-cząstek-stałych', 'is_3000-4000cm2', 'feature_szyberdach', 'feature_radio-niefabryczne', 'feature_hak', 'feature_alarm',
                  'feature_czujnik-zmierzchu', 'is_300-500KM', 'param_stan', 'feature_asystent-pasa-ruchu', 'feature_nawigacja-gps', 'feature_tuner-tv',
                  'is_over_500KM_and_2015', 'is_over_500KM_and_<50000km', 'is_over_500KM', 'is_5000000km', 'is_over_4000cm2', 'param_opłata-początkowa', 
                  'is_older_than_1960', 'param_zarejestrowany-jako-zabytek', 'is_1960-1970', 'param_matowy', 'is_EUR', 'is_Ferrari', 'is_Rolls-Royce',
                  'is_Bentley', 'is_AMG_GT', 'param_wartość-wykupu', 'is_R231 (2012-)', 'is_Ferrari_Warszawa', 'is_Multi_Cars', 'is_Luxury-Select-Car',
                  'is_Pn_Wlkp_Nowe_Miasto', 'is_Supra', 'przebieg_less_than_100km', 'feature_podgrzewana-przednia-szyba', 'is_less_30000km_and_2017',
                  'param_leasing', 'is_over_200000km']

In [34]:
#Podzielenie najlepszych cech na 3 części
def chunker_list(seq, size):
    return (seq[i::size] for i in range(size))

In [35]:
feats_a, feats_b, feats_c = list(chunker_list(feat_for_split, 3))

In [38]:
#Wybranie z każdej części najlepszych cech
#eli_feats(feats_a)
#eli_feats(feats_b)
eli_feats(feats_c)

Weight,Feature
0.1471 ± 0.0138,bool_feats_count
0.1354 ± 0.0123,param_przebieg
0.0832 ± 0.0076,param_przebieg_norm
0.0771 ± 0.0064,is_2011-2015
0.0648 ± 0.0059,is_BMW_or_Audi_or_Merc
0.0402 ± 0.0051,param_wersja_cat
0.0353 ± 0.0010,param_skrzynia-biegów_cat
0.0279 ± 0.0065,param_kod-silnika_cat
0.0167 ± 0.0025,is_luxury_car
0.0111 ± 0.0035,is_mid_car


In [39]:
final_check = ['param_rok-produkcji', 'param_moc_norm', 'param_marka_cat', 'param_uszkodzony', 'param_pojemność-skokowa_norm', 'is_over_2015', 'param_napęd_cat', 
               'finish_produce', 'seller_name_cat', 'param_liczba-miejsc', 'age_of_model', 'param_rodzaj-paliwa_cat', 'is_less_30000km_and_2017', 
               'feature_czujniki-parkowania-przednie', 'feature_elektrycznie-ustawiane-fotele', 'feature_światła-led', 'param_kolor_cat',
               'feature_czujniki-parkowania-tylne', 'feature_klimatyzacja-automatyczna', 'feature_klimatyzacja-dwustrefowa', 'is_100-300KM', 'param_faktura-vat',
               'is_2006-2010', 'is_vin', 'feature_czujnik-deszczu', 'feature_kurtyny-powietrzne', 'feature_asr-(kontrola-trakcji)', 'feature_czujnik-zmierzchu',
               'feature_dach-panoramiczny', 'feature_asystent-pasa-ruchu', 'feature_cd', 'feature_elektryczne-szyby-przednie', 'feature_klimatyzacja-czterostrefowa',
               'feature_poduszka-powietrzna-chroniąca-kolana', 'is_over_500KM_and_2015', 'start_produce', 'param_moc', 'key_feats_count', 'przebieg_less_than_100km',
               'param_typ_cat', 'param_pojemność-skokowa', 'param_model_cat', 'param_kraj_cat', 'is_100000-200000km', 'param_oferta-od', 'is_Audi', 'null_count', 
               'param_bezwypadkowy', 'feature_esp-(stabilizacja-toru-jazdy)', 'feature_system-start-stop', 'feature_tempomat', 'param_leasing',
               'param_kierownica-po-prawej-(anglik)', 'is_Klasa_S_G_GLS_GLE_E', 'param_zarejestrowany-w-polsce', 'feature_poduszka-powietrzna-kierowcy',
               'feature_alufelgi', 'feature_światła-xenonowe', 'param_metalik', 'feature_elektrycznie-ustawiane-lusterka', 'feature_bluetooth', 
               'param_możliwość-finansowania', 'feature_nawigacja-gps', 'feature_hak', 'feature_elektryczne-szyby-tylne', 'is_silver', 'is_over_500KM_and_<50000km',
               'param_miesięczna-rata', 'feature_regulowane-zawieszenie', 'is_3000-4000cm2', 'feature_elektrochromatyczne-lusterka-boczne', 
               'param_akryl-(niemetalizowany)', 'param_homologacja-ciężarowa', 'feature_ogrzewanie-postojowe', 'is_over_4000cm2', 'param_przebieg',
               'bool_feats_count', 'is_2011-2015', 'param_wersja_cat', 'is_BMW_or_Audi_or_Merc', 'param_skrzynia-biegów_cat', 'param_przebieg_norm',
               'param_kod-silnika_cat', 'feature_tapicerka-welurowa', 'param_serwisowany-w-aso', 'is_mid_car', 'feature_kamera-cofania', 'param_liczba-drzwi', 
               'is_luxury_car', 'is_Volkswagen', 'is_faktura_and_leasing', 'feature_immobilizer', 'error_features_true', 'feature_światła-do-jazdy-dziennej',
               'feature_klimatyzacja-manualna', 'feature_wielofunkcyjna-kierownica', 'feature_komputer-pokładowy', 'is_cheap_car', 'param_vat-marża', 
               'feature_przyciemniane-szyby', 'feature_gniazdo-aux', 'is_1960-1970', 'feature_alarm', 'feature_podgrzewane-przednie-siedzenia', 'is_over_500KM', 
               'feature_tapicerka-skórzana', 'feature_podgrzewana-przednia-szyba', 'is_100-20000km', 'is_Mercedes-Benz', 'feature_szyberdach', 'feature_centralny-zamek', 
               'feature_gniazdo-usb', 'is_BMW', 'param_stan', 'is_over_200000km', 'param_opłata-początkowa', 'feature_tuner-tv']

In [40]:
#Końcowe wybranie cech z 3 połączonych grup
eli_feats(final_check)

Weight,Feature
0.5149 ± 0.0178,param_rok-produkcji
0.0218 ± 0.0012,param_uszkodzony
0.0178 ± 0.0016,param_przebieg
0.0134 ± 0.0023,param_moc
0.0133 ± 0.0013,param_marka_cat
0.0130 ± 0.0020,param_typ_cat
0.0125 ± 0.0018,param_moc_norm
0.0108 ± 0.0017,is_BMW_or_Audi_or_Merc
0.0075 ± 0.0008,finish_produce
0.0069 ± 0.0008,param_napęd_cat


In [41]:
features = ['przebieg_less_than_100km', 'is_over_2015', 'param_rok-produkcji', 'param_moc_norm', 'param_przebieg', 'param_uszkodzony', 'param_typ_cat', 
            'is_BMW_or_Audi_or_Merc', 'param_napęd_cat', 'bool_feats_count', 'key_feats_count', 'param_marka_cat', 'param_pojemność-skokowa_norm', 
            'is_luxury_car', 'finish_produce', 'start_produce', 'param_model_cat', 'is_Volkswagen', 'param_pojemność-skokowa', 'param_liczba-miejsc', 
            'param_wersja_cat', 'is_mid_car', 'param_skrzynia-biegów_cat', 'age_of_model', 'feature_poduszka-powietrzna-kierowcy', 
            'param_kierownica-po-prawej-(anglik)', 'param_rodzaj-paliwa_cat', 'param_bezwypadkowy', 'param_kraj_cat', 'feature_tapicerka-welurowa',
            'param_faktura-vat', 'param_zarejestrowany-w-polsce', 'param_liczba-drzwi', 'param_moc', 'feature_alufelgi', 'is_vin', 'feature_alarm',
            'seller_name_cat', 'is_Audi', 'feature_gniazdo-usb', 'feature_światła-xenonowe', 'param_możliwość-finansowania', 'is_2011-2015', 'param_przebieg_norm',
            'is_100-300KM', 'is_cheap_car', 'param_leasing', 'is_less_30000km_and_2017', 'error_features_true', 'feature_gniazdo-aux', 'null_count', 
            'feature_klimatyzacja-manualna', 'param_kolor_cat', 'is_silver', 'feature_kurtyny-powietrzne', 'feature_komputer-pokładowy', 'param_kod-silnika_cat', 
            'feature_tempomat', 'feature_immobilizer', 'feature_esp-(stabilizacja-toru-jazdy)', 'is_Klasa_S_G_GLS_GLE_E', 'is_100000-200000km', 
            'feature_tapicerka-skórzana', 'is_Mercedes-Benz', 'feature_elektrochromatyczne-lusterka-boczne', 'param_oferta-od', 'is_2006-2010', 
            'param_akryl-(niemetalizowany)', 'feature_dach-panoramiczny', 'param_vat-marża', 'feature_przyciemniane-szyby', 'feature_szyberdach',
            'feature_czujniki-parkowania-tylne', 'feature_poduszka-powietrzna-chroniąca-kolana', 'feature_asr-(kontrola-trakcji)', 'feature_bluetooth',
            'feature_centralny-zamek', 'feature_klimatyzacja-dwustrefowa', 'feature_światła-do-jazdy-dziennej', 'is_faktura_and_leasing', 
            'feature_elektryczne-szyby-tylne', 'feature_asystent-pasa-ruchu', 'is_100-20000km', 'param_serwisowany-w-aso', 'feature_system-start-stop', 
            'feature_hak', 'feature_wielofunkcyjna-kierownica', 'param_opłata-początkowa', 'is_BMW', 'feature_klimatyzacja-automatyczna', 'is_3000-4000cm2', 
            'feature_światła-led', 'is_over_500KM_and_2015', 'is_over_500KM_and_<50000km', 'is_over_4000cm2', 'is_over_500KM', 'is_over_200000km']

In [44]:
train_light = train.sample(frac=0.1)
X_light = train_light[features].values
y_light_log = np.log1p(train_light['price_value'].values)

##Tuning hyperparameters

In [ ]:
def objective(space, early_stopping_rounds=12):
    
    ctb_params = {
        'iterations' : space['iterations'],
        'depth': int(space['depth']),
        'learning_rate': space['learning_rate'],
        'l2_leaf_reg': space['l2_leaf_reg'],
        'min_data_in_leaf': space['min_data_in_leaf'],
        'random_strength': space['random_strength'],
        'border_count' : space['border_count'],
        'verbose' : False,
        'early_stopping_rounds' : 15
    }
    
    cv = KFold(n_splits=3, shuffle=True, random_state=2018)
    scores = []
    for train_idx, test_idx in cv.split(X_light):
        model = ctb.CatBoostRegressor(**ctb_params)
        model.fit(X_light[train_idx], y_light_log[train_idx])
        y_pred = model.predict(X_light[test_idx])
    
        score = mean_absolute_error(y_light_log[test_idx], y_pred)
        scores.append(score)
    
    return{'loss':np.mean(scores), 'status': STATUS_OK }
    
space ={
    'iterations': hp.quniform('iterations', 25, 10000, 250),
    'depth': hp.quniform("depth", 1, 6, 1),
    'learning_rate' : hp.uniform('learning_rate', 0.05, 0.25),
    'l2_leaf_reg': hp.uniform('l2_leaf_reg', 3, 8),
    'min_data_in_leaf' : hp.quniform('min_data_in_leaf', 1, 50, 1),
    'random_strength' : hp.loguniform('random_strength', np.log(0.005), np.log(5)),
    'border_count': hp.quniform('border_count', 30, 220, 5)
}


trials = Trials()
best_params = fmin(fn=objective,
            space=space,
            algo=partial(tpe.suggest, n_startup_jobs=1),
            max_evals=50,
            trials=trials)

print("The best params: ", best_params)

In [46]:
ctb_params = {'border_count': 195, 'depth': 6, 'iterations': 29000, 'l2_leaf_reg': 5.885578634980554, 'learning_rate': 0.06301319304159513, 
              'min_data_in_leaf': 37, 'random_strength': 1.984062456758668}

In [47]:
model = ctb.CatBoostRegressor(**ctb_params)

In [48]:
X = train[features].values
y = train['price_value'].values
y_log = np.log1p(y)

In [49]:
model.fit(X, y_log, verbose=1000)

0:	learn: 0.9805485	total: 30.8ms	remaining: 14m 52s
1000:	learn: 0.2247069	total: 25.1s	remaining: 11m 42s
2000:	learn: 0.2033999	total: 49.5s	remaining: 11m 8s
3000:	learn: 0.1907798	total: 1m 13s	remaining: 10m 36s
4000:	learn: 0.1812390	total: 1m 37s	remaining: 10m 9s
5000:	learn: 0.1733891	total: 2m 2s	remaining: 9m 46s
6000:	learn: 0.1670785	total: 2m 26s	remaining: 9m 21s
7000:	learn: 0.1613145	total: 2m 50s	remaining: 8m 55s
8000:	learn: 0.1561949	total: 3m 14s	remaining: 8m 31s
9000:	learn: 0.1516189	total: 3m 39s	remaining: 8m 7s
10000:	learn: 0.1473409	total: 4m 3s	remaining: 7m 43s
11000:	learn: 0.1435031	total: 4m 28s	remaining: 7m 19s
12000:	learn: 0.1399587	total: 4m 53s	remaining: 6m 55s
13000:	learn: 0.1366813	total: 5m 18s	remaining: 6m 31s
14000:	learn: 0.1336712	total: 5m 43s	remaining: 6m 8s
15000:	learn: 0.1307934	total: 6m 8s	remaining: 5m 44s
16000:	learn: 0.1280545	total: 6m 33s	remaining: 5m 19s
17000:	learn: 0.1254914	total: 6m 58s	remaining: 4m 55s
18000:	le

In [50]:
X_pred = test[features].values

In [51]:
y_pred = np.expm1(model.predict(X_pred))

In [52]:
global_min = train.price_value.min()
y_pred[y_pred < global_min] = global_min

In [53]:
test['price_value'] = y_pred
#Konwersja cen w Euro
test['price_value'] = np.where(test['price_currency']=='EUR', test['price_value'].map(lambda x: x/4), test['price_value'].map(lambda x: x))

In [54]:
test[['car_id', 'price_value']].to_csv('catboost_49000_euro.csv', index=False)